<a href="https://colab.research.google.com/github/stmulugheta/Introduction-to-Web-Scraping/blob/main/run_food_tech_web_scraping_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Connect to gdrive (after you add a short to the share folder for FoodTech project)

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Install libraries

In [2]:
!pip install Scrapy
!pip install crochet

     |████████████████████████████████| 254 kB 5.2 MB/s 
     |████████████████████████████████| 3.5 MB 42.8 MB/s 
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 3.2 MB 32.0 MB/s 
     |████████████████████████████████| 3.1 MB 47.3 MB/s 
     |████████████████████████████████| 65 kB 2.5 MB/s 
     |████████████████████████████████| 251 kB 43.1 MB/s 
     |████████████████████████████████| 47 kB 3.5 MB/s 
     |████████████████████████████████| 74 kB 2.6 MB/s 
  Created wheel for protego: filename=Protego-0.1.16-py3-none-any.whl size=7782 sha256=fd454bc0d252e7f91f4a363444f3a194862875633caafd87be870cd6d9f31bd9
  Stored in directory: /root/.cache/pip/wheels/ca/44/01/3592ccfbcfaee4ab297c4097e6e9dbe1c7697e3531a39877ab
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=11515 sha256=f8a2e52a95767a04375c36f8867b16f234eaf005c0b5656873505ff345e43ffd
  Stored in directory: /root/.cache/pip/wheels/2d/18/21/3c6a732eaa

#### Import libraries

In [3]:
import os, sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [4]:
# Scrapy imports
# scrape webpage
import scrapy
from scrapy.crawler import CrawlerRunner
# text cleaning
import re
# Reactor restart
from crochet import setup, wait_for

#### Support functions

In [5]:
class WebscrapeToCsv(scrapy.Spider):
    """scrape first line of  quotes from website and save to json file"""
    name = "WebsiteToCsv"
    # start_urls = [
    #     'https://www.colruyt.be/',
    # ]
    start_urls = [
                  "https://www.aldi.be/"
    ]
    custom_settings = {
          'ITEM_PIPELINES': {
              '__main__.ExtractFirstLine': 1
          },
          'FEEDS': {
              "output.csv": {
                  'format': 'csv',
                  'overwrite': True
              }
          }
          # 'FEEDS': {
          #     'promo_data.csv': {
          #         'format': 'csv',
          #         'overwrite': True
          #     }
          # }
      }

    def parse(self, response):
        """parse data from urls"""
        for quote in response.css('div.mw-parser-output > ul > li'):
            yield {'quote': quote.extract()}


class ExtractFirstLine(object):
    def process_item(self, item, spider):
        """text processing"""
        lines = dict(item)[0].splitlines()
        first_line = self.__remove_html_tags__(lines[0])

        return {'0 - item': first_line}

    def __remove_html_tags__(self, text):
        """remove html tags from string"""
        html_tags = re.compile('<.*?>')
        return re.sub(html_tags, '', text)

@wait_for(10)
def run_spider(start_urls, output_path):
    """run spider with WebscrapeToCsv"""
    output_path = "output.csv"
    # call setup - only once
    setup()

    crawler = CrawlerRunner()
    d = crawler.crawl(WebscrapeToCsv)
    return d


#### Set path and get websites

In [10]:
current_run_path = ("https://drive.google.com/drive/u/0/folders/1VgQL3PVyF4ix_CAScRbk6jLfHydVNtL9")
os.chdir(current_run_path)


FileNotFoundError: ignored

In [ ]:
base_path = "/content/drive/MyDrive/Official_Folder_Omdena_FoodTech"

In [ ]:
websites_filename = os.path.join(base_path, "3_Data", "Retail Websites for scrapping.csv")

In [ ]:
# load data and extract only links for now
import chardet 
def get_char_encoding(path):
  charenc = ""
  rawdata = open(path, 'rb').read()
  result = chardet.detect(rawdata)
  charenc = result['encoding']
  print(charenc)
  return charenc

def read_data(path):
  df = None
  char_enc = get_char_encoding(path)
  with open(path, "r") as fs:
    df = pd.read_csv(path, encoding=char_enc)

  return df


In [ ]:
df_websites = read_data(websites_filename)
df_websites

utf-8


,retailer,link
0,Colruyt laagste prijzen,https://www.colruyt.be/
1,Okay,https://www.okay.be/okay/static/nl.shtml
2,Bio planet,https://www.bioplanet.be/
3,Alvo,https://www.alvo.be/
4,Spar (Colruyt),https://www.mijnspar.be/
5,Spar (Lambrechts),https://www.spar.be/
6,Delhaize,https://www.delhaize.be/
7,Louis Delhaize,https://louisdelhaize.be/
8,Carrefour,https://www.carrefour.be/
9,Aldi,https://www.aldi.be/


In [ ]:
# start_urls = [
#         'https://www.colruyt.be/',
#     ]

start_urls = df_websites["link"]
start_urls #[0]

0                      https://www.colruyt.be/
1     https://www.okay.be/okay/static/nl.shtml
2                    https://www.bioplanet.be/
3                         https://www.alvo.be/
4                     https://www.mijnspar.be/
5                         https://www.spar.be/
6                     https://www.delhaize.be/
7                    https://louisdelhaize.be/
8                    https://www.carrefour.be/
9                         https://www.aldi.be/
10                        https://www.lidl.be/
11                       https://www.makro.be/
12                       https://www.metro.be/
13                      https://www.farm.coop/
14                 https://www.jumbo.com/nl-be
15                          https://www.ah.be/
16                 https://www.intermarche.be/
17           https://www.supermarche-match.be/
Name: link, dtype: object

In [ ]:
run_spider()

In [ ]:
!pip install requests
!pip install beautifulsoup4


In [ ]:
!apt update
!apt install chromium-chromedriver

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download

In [ ]:
!which chromedriver

/usr/bin/chromedriver


In [ ]:
!pip install selenium

     |████████████████████████████████| 904 kB 14.2 MB/s 


In [ ]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
prefs = {
  "translate_whitelists": {"ru":"en"},
  "translate":{"enabled":"true"}
}

options.add_experimental_option("prefs", prefs)
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)

In [ ]:
import bs4 as bs
import pandas as pd
import datetime
import time        #note this line

# url_link = 'https://www.instagram.com/explore/tags/モデル'
url_link = start_urls[9]
wd.get(url_link)

time.sleep(8)                                          #note this as well moreover it should be after get method 
source = wd.execute_script("return document.body.innerHTML")



In [ ]:
soup = bs.BeautifulSoup(source,'lxml')
print(soup)
out_path = "/content/output"
if not os.path.exists(out_path):
  os.makedirs(out_path)
out_file = os.path.join(out_path, "sample.html")


In [ ]:
# curr_content = soup.find('span', class_='g47SY ') #.text
curr_content = soup.find('div', class_="mod-stage-theme__media mod-stage-theme__media--picture")
print(curr_content)

In [ ]:
def extract_elems_by_class(curr_soup, elem, cls):
  extracted_elems = []
  try:
    extracted_elems = curr_soup.find_all(elem, class_=cls)
  except Exception as ee:
    print(f"Unable to extract for element: {elem} with class: {cls}")

  return extracted_elems

In [ ]:
# for class="mod-offer-category__content"
curr_class="mod-offer-category__content"
html_elem = "div"
# promo_categories = soup.find_all('div', class_=curr_class)
promo_categories = extract_elems_by_class(soup, html_elem, curr_class)
print(promo_categories)

[<div class="mod-offer-category__content">
<h3 class="mod-offer-category__title">Thema-actie Frankrijk</h3>
<span class="mod-offer-category__tagline">sinds wo 06/10</span>
</div>, <div class="mod-offer-category__content">
<h3 class="mod-offer-category__title">Merkenpromo</h3>
<span class="mod-offer-category__tagline">sinds za 09/10</span>
</div>, <div class="mod-offer-category__content">
<h3 class="mod-offer-category__title">Verspromo</h3>
<span class="mod-offer-category__tagline">sinds ma 11/10</span>
</div>, <div class="mod-offer-category__content">
<h3 class="mod-offer-category__title">Promo food &amp; non-food</h3>
<span class="mod-offer-category__tagline">vanaf vr 15/10</span>
</div>, <div class="mod-offer-category__content">
<h3 class="mod-offer-category__title">Food &amp; non-food</h3>
<span class="mod-offer-category__tagline">vanaf vr 15/10</span>
</div>, <div class="mod-offer-category__content">
<h3 class="mod-offer-category__title">Badkamer</h3>
<span class="mod-offer-categor

In [ ]:
import lxml.html as lh

In [ ]:
promo_categories[0]

<div class="mod-offer-category__content">
<h3 class="mod-offer-category__title">Thema-actie Frankrijk</h3>
<span class="mod-offer-category__tagline">sinds wo 06/10</span>
</div>

In [ ]:
# tr_elements = promo_categories.xpath('//h3')
# doc = lh.fromstring(page.content)
# extracted_elements = [(lh.fromstring(item)).xpath('//h3') for item in promo_categories]
extracted_elements = [item for item in promo_categories]
# print(extracted_elements)
curr_class = "mod-offer-category__title"
# extracted_cats = soup.find_all('h3', class_=curr_class)
html_elem = "h3"
extracted_cats = extract_elems_by_class(soup, html_elem, curr_class)
promo_categories = [item.text for item in extracted_cats]
print(promo_categories)


['Thema-actie Frankrijk', 'Merkenpromo', 'Verspromo', 'Promo food & non-food', 'Food & non-food', 'Badkamer', 'Multimedia', 'Verspromo', 'Kleding', 'Overige', 'Promo food & non-food', 'Food & non-food', 'Kleding', 'Interieur']


In [ ]:
open(out_file, 'w').write(wd.page_source)

1907628